<a href="https://colab.research.google.com/github/hasobi/dos-classification/blob/master/ta_2_update_Mei2020_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from matplotlib import*
import matplotlib.pyplot as plt
from matplotlib.cm import register_cmap
from scipy import stats

import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

from scipy.stats import zscore

from sklearn.ensemble import ExtraTreesClassifier
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import cross_validate, ShuffleSplit

from sklearn.model_selection import KFold

from sklearn.neural_network import MLPClassifier

from sklearn.externals import joblib 

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Tugas Akhir a.k.a SKRIPSI/Tugas Akhir - UPDATE JANUARI 2020/dataset/KDDTrain_Test.csv')
# df = pd.read_csv('/content/drive/My Drive/Tugas Akhir a.k.a SKRIPSI/Tugas Akhir  LAMA/dataset/KDDTrain+_v1.csv')
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,20,normal
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,15,normal
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,19,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,21,normal
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148511,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,21,normal
148512,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,21,normal
148513,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,15,back
148514,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,21,normal


In [ ]:
# dicitionary untuk merubah tipe serangan menjadi lebih umum seperti Normal, DoS, Probe, U2R, dan R2L

attack_dict={ #DOS
    'back' : 'dos',
    'land' : 'dos',
    'neptune' : 'dos',
    'pod' : 'dos',
    'smurf' : 'dos',
    'teardrop' : 'dos',
    'apache2' : 'dos',
    'udpstorm' : 'dos',
    'processtable' : 'dos',
    'worm' : 'dos',
    'mailbomb' : 'dos',
    
    #probe
    'satan' : 'probe',
    'ipsweep' : 'probe',
    'nmap' : 'probe',
    'portsweep' : 'probe',
    'mscan' : 'probe',
    'saint' : 'probe',
    
    #R2L
    'guess_passwd' : 'r2l',
    'ftp_write' : 'r2l',
    'imap' : 'r2l',
    'phf' : 'r2l',
    'multihop' : 'r2l',
    'warezmaster' : 'r2l',
    'warezclient' : 'r2l',
    'spy' : 'r2l',
    'xlock' : 'r2l',
    'xsnoop': 'r2l',
    'snmpguess' : 'r2l',
    'snmpgetattack' : 'r2l',
    'httptunnel' : 'r2l',
    'sendmail' : 'r2l',
    'named' : 'r2l',
    
    #U2R
    'buffer_overflow' : 'u2r',
    'loadmodule' : 'u2r',
    'rootkit' : 'u2r',
    'perl' : 'u2r',
    'sqlattack' : 'u2r',
    'xterm' : 'u2r',
    'ps' : 'u2r'
}

protocol_dict ={
    "udp" : 0,
    "tcp" : 1,
    "icmp": 2
}

service_dict={
    "Z39_50" : 0,
    "X11" : 1,
    "whois" : 2,
    "vmnet" : 3,
    "uucp_path" : 4,
    "uucp" : 5,
    "urp_i" : 6,
    "urh_i" : 7,
    "time" : 8,
    "tim_i" : 9,
    "tftp_u" : 10,
    "telnet" : 11,
    "systat" : 12,
    "supdup" : 13,
    "sunrpc" : 14,
    "ssh" : 15,
    "sql_net" : 16,
    "smtp" : 17,
    "shell" : 18,
    "rje" : 19,
    "remote_job" : 20,
    "red_i" : 21,
    "private" : 22,
    "printer" : 23,
    "pop_3" : 24,
    "pop_2" : 25,
    "pm_dump" : 26,
    "other" : 27,
    "ntp_u" : 28,
    "nntp" : 29,
    "nnsp" : 30,
    "netstat" : 31,
    "netbios_ssn" : 32,
    "netbios_ns" : 33,
    "netbios_dgm" : 34,
    "name" : 35,
    "mtp" : 36,
    "login" : 37,
    "link" : 38,
    "ldap" : 39,
    "kshell" : 40,
    "klogin" : 41,
    "iso_tsap" : 42,
    "IRC" : 43,
    "imap4" : 44,
    "http_8001" : 45,
    "http_443" : 46,
    "http_2784" : 47,
    "http" : 48,
    "hostnames" : 49,
    "harvest" : 50,
    "gopher" : 51,
    "ftp_data" : 52,
    "ftp" : 53,
    "finger" :  54,
    "exec" : 55,
    "efs" : 56,
    "ecr_i" : 57,
    "eco_i" : 58,
    "echo" : 59,
    "domain_u" : 60,
    "domain" : 61,
    "discard" : 62,
    "daytime" : 63,
    "ctf" : 64,
    "csnet_ns" : 65,
    "courier" : 66,
    "bgp" : 67,
    "auth" : 68,
    "aol" : 69

}

flag_dict={
    "SH" : 0,
    "SF" : 1,
    "S3" : 2,
    "S2" : 3,
    "S1" : 4,
    "S0" : 5,
    "RSTR" : 6,
    "RSTOS0" : 7,
    "RSTO" : 8,
    "REJ" : 9,
    "OTH" : 10

}



df.replace(flag_dict, inplace = True)
df.replace(service_dict, inplace = True)
df.replace(protocol_dict, inplace = True)
df.replace(attack_dict, inplace = True)

In [ ]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,1,52,1,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,20,normal
1,0,0,27,1,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,15,normal
2,0,1,22,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,19,dos
3,0,1,48,1,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,21,normal
4,0,1,48,1,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,normal


In [ ]:
df.shape

(148516, 43)

In [ ]:
df.type_attack.unique()

array(['normal', 'dos', 'r2l', 'probe', 'u2r'], dtype=object)

In [ ]:
# Data yang digunakan untuk pembuatan model klasifikasi DoS & normal

data_normaldos = df.loc[df['type_attack'].isin(['normal','dos'])]
data_normaldos.shape

(130440, 43)

In [ ]:
#View jumlah data normal

nrml = data_normaldos.loc[data_normaldos['type_attack'].isin(['normal'])]
nrml.shape

(77053, 43)

In [ ]:
#View jumlah data DoS

doss = data_normaldos.loc[data_normaldos['type_attack'].isin(['dos'])]
doss.shape

(53387, 43)

In [ ]:
print(df.loc[df['type_attack'].isin(['probe'])].shape)

(14077, 43)


In [ ]:
print(df.loc[df['type_attack'].isin(['u2r'])].shape)

(119, 43)


In [ ]:
print(df.loc[df['type_attack'].isin(['r2l'])].shape)

(3880, 43)


In [ ]:
X = data_normaldos.loc[:, :'dst_host_srv_rerror_rate']
y = data_normaldos.loc[:, 'type_attack']

y = y.astype(str)

temp = set(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size =0.8, random_state=5)
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state=5)



clf = MLPClassifier(random_state = 5)

In [ ]:
X_train.shape

(104352, 41)

In [ ]:
y_train.shape

(104352,)

In [ ]:
dataTrain = pd.concat([X_train,y_train], axis = 1)

In [ ]:
dataTrain.shape

(104352, 42)

In [ ]:
# Convert data training ke CSV
dataTrain.to_csv('dataTrain.csv',index=False)

In [ ]:
trainn = pd.read_csv('/content/dataTrain.csv')
trainn

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_attack
0,0,1,22,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,142,4,0.0,0.0,1.00,1.00,0.03,0.06,0.00,255,4,0.02,0.05,0.00,0.00,0.00,0.00,1.00,1.00,dos
1,0,1,48,1,244,870,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,15,0.0,0.0,0.00,0.00,1.00,0.00,0.13,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
2,0,1,22,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,138,22,1.0,1.0,0.00,0.00,0.16,0.05,0.00,255,22,0.09,0.06,0.00,0.00,1.00,1.00,0.00,0.00,dos
3,0,1,48,1,368,1871,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,20,0.0,0.0,0.00,0.00,1.00,0.00,0.30,70,244,1.00,0.00,0.01,0.08,0.00,0.00,0.20,0.60,normal
4,7582,1,11,1,0,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.00,0.00,0.00,255,178,0.70,0.02,0.00,0.00,0.44,0.43,0.01,0.01,dos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,0,0,60,1,45,133,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,191,190,0.0,0.0,0.00,0.00,0.99,0.01,0.00,80,47,0.59,0.03,0.01,0.00,0.00,0.00,0.00,0.00,normal
104348,0,1,22,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,232,20,0.0,0.0,1.00,1.00,0.09,0.07,0.00,255,20,0.08,0.08,0.00,0.00,0.00,0.00,1.00,1.00,dos
104349,0,1,67,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269,7,1.0,1.0,0.00,0.00,0.03,0.05,0.00,255,7,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,dos
104350,0,1,48,6,53208,7300,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,3,0.0,0.0,0.25,0.33,0.75,0.50,0.00,72,72,1.00,0.00,0.01,0.00,0.00,0.00,0.04,0.04,dos


In [ ]:
start = time.time()

clf.fit(X_train, y_train)

stop = time.time()

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
print(f"training time : {stop - start}s")

training time : 169.9430651664734s


In [ ]:
# Save model

# joblib.dump(clf, 'model1.pkl') 
joblib.dump(clf, 'model_base.pkl') 

['model_base.pkl']

In [ ]:
pred = clf.predict(X_test)

accuracy_score(y_test,pred)

0.9983134007973015

In [ ]:
y_test.shape

(26088,)

In [ ]:
clf = MLPClassifier(random_state = 5)
scoring = {'accuracy': make_scorer(accuracy_score)}
cv = ShuffleSplit(n_splits=3, test_size=0.2,train_size = 0.8, random_state=5)

In [ ]:
start = time.time()

scores = cross_validate(clf, X, y, cv=cv, scoring=scoring, return_train_score=True)

stop = time.time()

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
print(f"cross validation time : {stop - start}s")

cross validation time : 544.1082384586334s


In [ ]:
print(scores)

{'fit_time': array([170.86572051, 190.0606997 , 180.2291069 ]), 'score_time': array([0.19115782, 0.20633769, 0.2118206 ]), 'test_accuracy': array([0.9983134 , 0.99689512, 0.99513186]), 'train_accuracy': array([0.99852423, 0.99746052, 0.99608057])}


# Tes dataset log

In [ ]:
# df_test = pd.read_excel('/content/drive/My Drive/Tugas Akhir a.k.a SKRIPSI/Tugas Akhir - UPDATE JANUARI 2020/dataset/datatest_fromsnortlog.xlsx')
df_test = pd.read_csv('/content/drive/My Drive/Tugas Akhir a.k.a SKRIPSI/Tugas Akhir - UPDATE MARET 2020/datatest-fromsnortlog-v4.csv')


df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,icmp,http,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
1,0,tcp,http,REJ,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
2,0,icmp,http,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
3,0,tcp,http,REJ,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
4,0,icmp,http,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,0,tcp,ftp_data,SF,14052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.00,0.00,0.0,255,11,0.04,0.01,0.04,0.00,0.0,0.0,0.0,0.0,21,normal
1846,0,tcp,uucp,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103,3,1.0,1.0,0.0,0.0,0.03,0.07,0.0,255,12,0.05,0.07,0.00,0.00,1.0,1.0,0.0,0.0,19,dos
1847,0,tcp,http,SF,303,831,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.00,0.00,0.0,119,255,1.00,0.00,0.01,0.02,0.0,0.0,0.0,0.0,21,normal
1848,0,tcp,http,SF,279,3869,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.00,0.00,0.0,72,255,1.00,0.00,0.01,0.06,0.0,0.0,0.0,0.0,21,normal


In [ ]:
# dicitionary untuk merubah tipe serangan menjadi lebih umum seperti Normal, DoS, Probe, U2R, dan R2L

attack_dict={ #DOS
    'back' : 'dos',
    'land' : 'dos',
    'neptune' : 'dos',
    'pod' : 'dos',
    'smurf' : 'dos',
    'teardrop' : 'dos',
    'apache2' : 'dos',
    'udpstorm' : 'dos',
    'processtable' : 'dos',
    'worm' : 'dos',
    'mailbomb' : 'dos',
    
    #probe
    'satan' : 'probe',
    'ipsweep' : 'probe',
    'nmap' : 'probe',
    'portsweep' : 'probe',
    'mscan' : 'probe',
    'saint' : 'probe',
    
    #R2L
    'guess_passwd' : 'r2l',
    'ftp_write' : 'r2l',
    'imap' : 'r2l',
    'phf' : 'r2l',
    'multihop' : 'r2l',
    'warezmaster' : 'r2l',
    'warezclient' : 'r2l',
    'spy' : 'r2l',
    'xlock' : 'r2l',
    'xsnoop': 'r2l',
    'snmpguess' : 'r2l',
    'snmpgetattack' : 'r2l',
    'httptunnel' : 'r2l',
    'sendmail' : 'r2l',
    'named' : 'r2l',
    
    #U2R
    'buffer_overflow' : 'u2r',
    'loadmodule' : 'u2r',
    'rootkit' : 'u2r',
    'perl' : 'u2r',
    'sqlattack' : 'u2r',
    'xterm' : 'u2r',
    'ps' : 'u2r'
}

protocol_dict ={
    "udp" : 0,
    "tcp" : 1,
    "icmp": 2
}

service_dict={
    "Z39_50" : 0,
    "X11" : 1,
    "whois" : 2,
    "vmnet" : 3,
    "uucp_path" : 4,
    "uucp" : 5,
    "urp_i" : 6,
    "urh_i" : 7,
    "time" : 8,
    "tim_i" : 9,
    "tftp_u" : 10,
    "telnet" : 11,
    "systat" : 12,
    "supdup" : 13,
    "sunrpc" : 14,
    "ssh" : 15,
    "sql_net" : 16,
    "smtp" : 17,
    "shell" : 18,
    "rje" : 19,
    "remote_job" : 20,
    "red_i" : 21,
    "private" : 22,
    "printer" : 23,
    "pop_3" : 24,
    "pop_2" : 25,
    "pm_dump" : 26,
    "other" : 27,
    "ntp_u" : 28,
    "nntp" : 29,
    "nnsp" : 30,
    "netstat" : 31,
    "netbios_ssn" : 32,
    "netbios_ns" : 33,
    "netbios_dgm" : 34,
    "name" : 35,
    "mtp" : 36,
    "login" : 37,
    "link" : 38,
    "ldap" : 39,
    "kshell" : 40,
    "klogin" : 41,
    "iso_tsap" : 42,
    "IRC" : 43,
    "imap4" : 44,
    "http_8001" : 45,
    "http_443" : 46,
    "http_2784" : 47,
    "http" : 48,
    "hostnames" : 49,
    "harvest" : 50,
    "gopher" : 51,
    "ftp_data" : 52,
    "ftp" : 53,
    "finger" :  54,
    "exec" : 55,
    "efs" : 56,
    "ecr_i" : 57,
    "eco_i" : 58,
    "echo" : 59,
    "domain_u" : 60,
    "domain" : 61,
    "discard" : 62,
    "daytime" : 63,
    "ctf" : 64,
    "csnet_ns" : 65,
    "courier" : 66,
    "bgp" : 67,
    "auth" : 68,
    "aol" : 69

}

flag_dict={
    "SH" : 0,
    "SF" : 1,
    "S3" : 2,
    "S2" : 3,
    "S1" : 4,
    "S0" : 5,
    "RSTR" : 6,
    "RSTOS0" : 7,
    "RSTO" : 8,
    "REJ" : 9,
    "OTH" : 10

}



df_test.replace(flag_dict, inplace = True)
df_test.replace(service_dict, inplace = True)
df_test.replace(protocol_dict, inplace = True)
df_test.replace(attack_dict, inplace = True)

df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,2,48,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
1,0,1,48,9,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
2,0,2,48,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
3,0,1,48,9,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
4,0,2,48,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,0,1,52,1,14052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.00,0.00,0.0,255,11,0.04,0.01,0.04,0.00,0.0,0.0,0.0,0.0,21,normal
1846,0,1,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103,3,1.0,1.0,0.0,0.0,0.03,0.07,0.0,255,12,0.05,0.07,0.00,0.00,1.0,1.0,0.0,0.0,19,dos
1847,0,1,48,1,303,831,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.00,0.00,0.0,119,255,1.00,0.00,0.01,0.02,0.0,0.0,0.0,0.0,21,normal
1848,0,1,48,1,279,3869,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.00,0.00,0.0,72,255,1.00,0.00,0.01,0.06,0.0,0.0,0.0,0.0,21,normal


In [ ]:
X_test= df_test.loc[:, :'dst_host_srv_rerror_rate']
y_test = df_test.loc[:, 'type_attack']

y = y.astype(str)

temp = set(y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, train_size =0.3, random_state=5)
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state=5)



# clf = MLPClassifier(random_state = 5)

In [ ]:
clf = joblib.load('model_base.pkl')
# clf = MLPClassifier(random_state = 5)


In [ ]:
pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test,pred)

0.9989189189189189

# Seleksi fitur

**Buat model dari NSL KDD**

In [ ]:
df2 = pd.read_csv('/content/drive/My Drive/Tugas Akhir a.k.a SKRIPSI/Tugas Akhir - UPDATE JANUARI 2020/dataset/KDDTrain_Test.csv')
df2.head

<bound method NDFrame.head of         duration protocol_type  ... num_packet type_attack
0              0           tcp  ...         20      normal
1              0           udp  ...         15      normal
2              0           tcp  ...         19     neptune
3              0           tcp  ...         21      normal
4              0           tcp  ...         21      normal
...          ...           ...  ...        ...         ...
148511         0           tcp  ...         21      normal
148512         0           tcp  ...         21      normal
148513         0           tcp  ...         15        back
148514         0           udp  ...         21      normal
148515         0           tcp  ...         14       mscan

[148516 rows x 43 columns]>

In [ ]:
# dicitionary untuk merubah tipe serangan menjadi lebih umum seperti Normal, DoS, Probe, U2R, dan R2L

attack_dict={ #DOS
    'back' : 'dos',
    'land' : 'dos',
    'neptune' : 'dos',
    'pod' : 'dos',
    'smurf' : 'dos',
    'teardrop' : 'dos',
    'apache2' : 'dos',
    'udpstorm' : 'dos',
    'processtable' : 'dos',
    'worm' : 'dos',
    'mailbomb' : 'dos',
    
    #probe
    'satan' : 'probe',
    'ipsweep' : 'probe',
    'nmap' : 'probe',
    'portsweep' : 'probe',
    'mscan' : 'probe',
    'saint' : 'probe',
    
    #R2L
    'guess_passwd' : 'r2l',
    'ftp_write' : 'r2l',
    'imap' : 'r2l',
    'phf' : 'r2l',
    'multihop' : 'r2l',
    'warezmaster' : 'r2l',
    'warezclient' : 'r2l',
    'spy' : 'r2l',
    'xlock' : 'r2l',
    'xsnoop': 'r2l',
    'snmpguess' : 'r2l',
    'snmpgetattack' : 'r2l',
    'httptunnel' : 'r2l',
    'sendmail' : 'r2l',
    'named' : 'r2l',
    
    #U2R
    'buffer_overflow' : 'u2r',
    'loadmodule' : 'u2r',
    'rootkit' : 'u2r',
    'perl' : 'u2r',
    'sqlattack' : 'u2r',
    'xterm' : 'u2r',
    'ps' : 'u2r'
}

protocol_dict ={
    "udp" : 0,
    "tcp" : 1,
    "icmp": 2
}

service_dict={
    "Z39_50" : 0,
    "X11" : 1,
    "whois" : 2,
    "vmnet" : 3,
    "uucp_path" : 4,
    "uucp" : 5,
    "urp_i" : 6,
    "urh_i" : 7,
    "time" : 8,
    "tim_i" : 9,
    "tftp_u" : 10,
    "telnet" : 11,
    "systat" : 12,
    "supdup" : 13,
    "sunrpc" : 14,
    "ssh" : 15,
    "sql_net" : 16,
    "smtp" : 17,
    "shell" : 18,
    "rje" : 19,
    "remote_job" : 20,
    "red_i" : 21,
    "private" : 22,
    "printer" : 23,
    "pop_3" : 24,
    "pop_2" : 25,
    "pm_dump" : 26,
    "other" : 27,
    "ntp_u" : 28,
    "nntp" : 29,
    "nnsp" : 30,
    "netstat" : 31,
    "netbios_ssn" : 32,
    "netbios_ns" : 33,
    "netbios_dgm" : 34,
    "name" : 35,
    "mtp" : 36,
    "login" : 37,
    "link" : 38,
    "ldap" : 39,
    "kshell" : 40,
    "klogin" : 41,
    "iso_tsap" : 42,
    "IRC" : 43,
    "imap4" : 44,
    "http_8001" : 45,
    "http_443" : 46,
    "http_2784" : 47,
    "http" : 48,
    "hostnames" : 49,
    "harvest" : 50,
    "gopher" : 51,
    "ftp_data" : 52,
    "ftp" : 53,
    "finger" :  54,
    "exec" : 55,
    "efs" : 56,
    "ecr_i" : 57,
    "eco_i" : 58,
    "echo" : 59,
    "domain_u" : 60,
    "domain" : 61,
    "discard" : 62,
    "daytime" : 63,
    "ctf" : 64,
    "csnet_ns" : 65,
    "courier" : 66,
    "bgp" : 67,
    "auth" : 68,
    "aol" : 69

}

flag_dict={
    "SH" : 0,
    "SF" : 1,
    "S3" : 2,
    "S2" : 3,
    "S1" : 4,
    "S0" : 5,
    "RSTR" : 6,
    "RSTOS0" : 7,
    "RSTO" : 8,
    "REJ" : 9,
    "OTH" : 10

}



df2.replace(flag_dict, inplace = True)
df2.replace(service_dict, inplace = True)
df2.replace(protocol_dict, inplace = True)
df2.replace(attack_dict, inplace = True)

df2

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,1,52,1,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,20,normal
1,0,0,27,1,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,15,normal
2,0,1,22,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,19,dos
3,0,1,48,1,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,21,normal
4,0,1,48,1,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148511,0,1,17,1,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,21,normal
148512,0,1,48,1,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,21,normal
148513,0,1,48,1,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,15,dos
148514,0,0,60,1,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,21,normal


In [ ]:
data_normaldos2 = df2.loc[df['type_attack'].isin(['normal','dos'])]
data_normaldos2.shape

(130440, 43)

In [ ]:
# fitur 2, 3, 4, 5, 7, 8, 10, 13, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34,35, 36, 38, 39, 40, 41
# delete 1,6,11,12,14,15,16,17 (num_file_creations),18,19,20,21,22,32,,37
data_normaldos2.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,1,52,1,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,20,normal
1,0,0,27,1,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,15,normal
2,0,1,22,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,19,dos
3,0,1,48,1,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,21,normal
4,0,1,48,1,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,normal


In [ ]:
## Hapus fitur yang dianggap tidak significant terhadap klasifikasi serangan

df2.drop(['duration', 
             'dst_bytes', 
             'num_failed_logins',
             'logged_in',
             'root_shell',
             'su_attempted',
             'num_root',
             'num_file_creations',
             'num_shells',
             'num_access_files',
             'num_outbound_cmds',
             'is_host_login',
             'is_guest_login',
             'dst_host_count',
             'dst_host_srv_diff_host_rate'
             ], axis=1, inplace=True)

df2.head()

,protocol_type,service,flag,src_bytes,land,wrong_fragment,urgent,hot,num_compromised,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,1,52,1,491,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,25,0.17,0.03,0.17,0.00,0.00,0.05,0.00,20,normal
1,0,27,1,146,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,15,normal
2,1,22,5,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,26,0.10,0.05,0.00,1.00,1.00,0.00,0.00,19,dos
3,1,48,1,232,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,255,1.00,0.00,0.03,0.03,0.01,0.00,0.01,21,normal
4,1,48,1,199,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,21,normal


In [ ]:
df2.shape

(148516, 28)

In [ ]:
data_normaldos3 = df2.loc[df['type_attack'].isin(['normal','dos'])]
data_normaldos3.shape

(130440, 28)

In [ ]:
X = data_normaldos3.loc[:, :'dst_host_srv_rerror_rate']
y = data_normaldos3.loc[:, 'type_attack']

y = y.astype(str)

temp = set(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size =0.8, random_state=5)
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state=5)


In [ ]:
# clf = joblib.load('model_base.pkl')
clf = MLPClassifier(random_state = 5)


In [ ]:
start = time.time()

clf.fit(X_train, y_train)

stop = time.time()

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
# Save model

# joblib.dump(clf, 'model1.pkl') 
joblib.dump(clf, 'model_seleksi.pkl') 

['model_seleksi.pkl']

In [ ]:
print(f"training time : {stop - start}s")

training time : 200.0652847290039s


In [ ]:
pred = clf.predict(X_test)

accuracy_score(y_test,pred)

0.9933302667893285

In [ ]:
clf = MLPClassifier(random_state = 5)
scoring = {'accuracy': make_scorer(accuracy_score)}
cv = ShuffleSplit(n_splits=3, test_size=0.2,train_size = 0.8, random_state=5)

In [ ]:
start = time.time()

scores = cross_validate(clf, X, y, cv=cv, scoring=scoring, return_train_score=True)

stop = time.time()

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
print(f"cross validation time : {stop - start}s")

cross validation time : 570.8877372741699s


In [ ]:
print(scores)

{'fit_time': array([202.98941398, 182.17197919, 182.02751541]), 'score_time': array([0.29441381, 0.2360034 , 0.27634811]), 'test_accuracy': array([0.99333027, 0.99463355, 0.99425023]), 'train_accuracy': array([0.9942119 , 0.99540977, 0.99438439])}


# Tes model Seleksi Fitur pada datalog

In [ ]:
df_test2 = pd.read_csv('/content/drive/My Drive/Tugas Akhir a.k.a SKRIPSI/Tugas Akhir - UPDATE MARET 2020/datatest-fromsnortlog-v4.csv')
df_test2.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,icmp,http,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
1,0,tcp,http,REJ,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
2,0,icmp,http,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
3,0,tcp,http,REJ,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
4,0,icmp,http,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos


In [ ]:
# dicitionary untuk merubah tipe serangan menjadi lebih umum seperti Normal, DoS, Probe, U2R, dan R2L

attack_dict={ #DOS
    'back' : 'dos',
    'land' : 'dos',
    'neptune' : 'dos',
    'pod' : 'dos',
    'smurf' : 'dos',
    'teardrop' : 'dos',
    'apache2' : 'dos',
    'udpstorm' : 'dos',
    'processtable' : 'dos',
    'worm' : 'dos',
    'mailbomb' : 'dos',
    
    #probe
    'satan' : 'probe',
    'ipsweep' : 'probe',
    'nmap' : 'probe',
    'portsweep' : 'probe',
    'mscan' : 'probe',
    'saint' : 'probe',
    
    #R2L
    'guess_passwd' : 'r2l',
    'ftp_write' : 'r2l',
    'imap' : 'r2l',
    'phf' : 'r2l',
    'multihop' : 'r2l',
    'warezmaster' : 'r2l',
    'warezclient' : 'r2l',
    'spy' : 'r2l',
    'xlock' : 'r2l',
    'xsnoop': 'r2l',
    'snmpguess' : 'r2l',
    'snmpgetattack' : 'r2l',
    'httptunnel' : 'r2l',
    'sendmail' : 'r2l',
    'named' : 'r2l',
    
    #U2R
    'buffer_overflow' : 'u2r',
    'loadmodule' : 'u2r',
    'rootkit' : 'u2r',
    'perl' : 'u2r',
    'sqlattack' : 'u2r',
    'xterm' : 'u2r',
    'ps' : 'u2r'
}

protocol_dict ={
    "udp" : 0,
    "tcp" : 1,
    "icmp": 2
}

service_dict={
    "Z39_50" : 0,
    "X11" : 1,
    "whois" : 2,
    "vmnet" : 3,
    "uucp_path" : 4,
    "uucp" : 5,
    "urp_i" : 6,
    "urh_i" : 7,
    "time" : 8,
    "tim_i" : 9,
    "tftp_u" : 10,
    "telnet" : 11,
    "systat" : 12,
    "supdup" : 13,
    "sunrpc" : 14,
    "ssh" : 15,
    "sql_net" : 16,
    "smtp" : 17,
    "shell" : 18,
    "rje" : 19,
    "remote_job" : 20,
    "red_i" : 21,
    "private" : 22,
    "printer" : 23,
    "pop_3" : 24,
    "pop_2" : 25,
    "pm_dump" : 26,
    "other" : 27,
    "ntp_u" : 28,
    "nntp" : 29,
    "nnsp" : 30,
    "netstat" : 31,
    "netbios_ssn" : 32,
    "netbios_ns" : 33,
    "netbios_dgm" : 34,
    "name" : 35,
    "mtp" : 36,
    "login" : 37,
    "link" : 38,
    "ldap" : 39,
    "kshell" : 40,
    "klogin" : 41,
    "iso_tsap" : 42,
    "IRC" : 43,
    "imap4" : 44,
    "http_8001" : 45,
    "http_443" : 46,
    "http_2784" : 47,
    "http" : 48,
    "hostnames" : 49,
    "harvest" : 50,
    "gopher" : 51,
    "ftp_data" : 52,
    "ftp" : 53,
    "finger" :  54,
    "exec" : 55,
    "efs" : 56,
    "ecr_i" : 57,
    "eco_i" : 58,
    "echo" : 59,
    "domain_u" : 60,
    "domain" : 61,
    "discard" : 62,
    "daytime" : 63,
    "ctf" : 64,
    "csnet_ns" : 65,
    "courier" : 66,
    "bgp" : 67,
    "auth" : 68,
    "aol" : 69

}

flag_dict={
    "SH" : 0,
    "SF" : 1,
    "S3" : 2,
    "S2" : 3,
    "S1" : 4,
    "S0" : 5,
    "RSTR" : 6,
    "RSTOS0" : 7,
    "RSTO" : 8,
    "REJ" : 9,
    "OTH" : 10

}



df_test2.replace(flag_dict, inplace = True)
df_test2.replace(service_dict, inplace = True)
df_test2.replace(protocol_dict, inplace = True)
df_test2.replace(attack_dict, inplace = True)

df_test2.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,2,48,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
1,0,1,48,9,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
2,0,2,48,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
3,0,1,48,9,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
4,0,2,48,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos


In [ ]:
df_test2.drop(['duration', 
             'dst_bytes', 
             'num_failed_logins',
             'logged_in',
             'root_shell',
             'su_attempted',
             'num_root',
             'num_file_creations',
             'num_shells',
             'num_access_files',
             'num_outbound_cmds',
             'is_host_login',
             'is_guest_login',
             'dst_host_count',
             'dst_host_srv_diff_host_rate'
             ], axis=1, inplace=True)

df_test2.head()

,protocol_type,service,flag,src_bytes,land,wrong_fragment,urgent,hot,num_compromised,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,2,48,9,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,20,0.08,0.07,0.00,0.0,0.0,1.0,1.0,20,dos
1,1,48,9,64,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,250,1.00,0.00,0.33,0.0,0.0,1.0,1.0,21,normal
2,2,48,9,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,20,0.08,0.07,0.00,0.0,0.0,1.0,1.0,20,dos
3,1,48,9,64,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.0,250,1.00,0.00,0.33,0.0,0.0,1.0,1.0,21,normal
4,2,48,9,0,0,0,0,0,0,299,20,0.0,0.0,1.0,1.0,0.07,0.06,0.0,20,0.08,0.07,0.00,0.0,0.0,1.0,1.0,20,dos


In [ ]:
df_test2.shape

(1850, 28)

In [ ]:
X_test = df_test2.loc[:, :'dst_host_srv_rerror_rate']
y_test = df_test2.loc[:, 'type_attack']

y = y.astype(str)

temp = set(y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, train_size =0.5, random_state=5)
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state=5)


In [ ]:
clf = joblib.load('/content/model_seleksi.pkl')

In [ ]:
pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test,pred)

0.9956756756756757

In [ ]:
clf = MLPClassifier(random_state = 5)
scoring = {'accuracy': make_scorer(accuracy_score)}
cv = ShuffleSplit(n_splits=3, test_size=0.2,train_size = 0.8, random_state=5)

In [ ]:
start = time.time()

scores = cross_validate(clf, X, y, cv=cv, scoring=scoring, return_train_score=True)

stop = time.time()

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
print(f"cross validation time : {stop - start}s")

cross validation time : 561.2097795009613s


In [ ]:
print(scores)

{'fit_time': array([195.73429179, 180.44895291, 181.4157331 ]), 'score_time': array([0.30987763, 0.23552895, 0.27333307]), 'test_accuracy': array([0.99333027, 0.99463355, 0.99425023]), 'train_accuracy': array([0.9942119 , 0.99540977, 0.99438439])}


# Tes model dengan data yang sengaja dirusak

In [ ]:
df_test_datar = pd.read_excel('/content/drive/My Drive/Tugas Akhir a.k.a SKRIPSI/Tugas Akhir - UPDATE MARET 2020/datatest-fromsnortlog-sengajadirusak.xlsx')

In [ ]:
# dicitionary untuk merubah tipe serangan menjadi lebih umum seperti Normal, DoS, Probe, U2R, dan R2L

attack_dict={ #DOS
    'back' : 'dos',
    'land' : 'dos',
    'neptune' : 'dos',
    'pod' : 'dos',
    'smurf' : 'dos',
    'teardrop' : 'dos',
    'apache2' : 'dos',
    'udpstorm' : 'dos',
    'processtable' : 'dos',
    'worm' : 'dos',
    'mailbomb' : 'dos',
    
    #probe
    'satan' : 'probe',
    'ipsweep' : 'probe',
    'nmap' : 'probe',
    'portsweep' : 'probe',
    'mscan' : 'probe',
    'saint' : 'probe',
    
    #R2L
    'guess_passwd' : 'r2l',
    'ftp_write' : 'r2l',
    'imap' : 'r2l',
    'phf' : 'r2l',
    'multihop' : 'r2l',
    'warezmaster' : 'r2l',
    'warezclient' : 'r2l',
    'spy' : 'r2l',
    'xlock' : 'r2l',
    'xsnoop': 'r2l',
    'snmpguess' : 'r2l',
    'snmpgetattack' : 'r2l',
    'httptunnel' : 'r2l',
    'sendmail' : 'r2l',
    'named' : 'r2l',
    
    #U2R
    'buffer_overflow' : 'u2r',
    'loadmodule' : 'u2r',
    'rootkit' : 'u2r',
    'perl' : 'u2r',
    'sqlattack' : 'u2r',
    'xterm' : 'u2r',
    'ps' : 'u2r'
}

protocol_dict ={
    "udp" : 0,
    "tcp" : 1,
    "icmp": 2
}

service_dict={
    "Z39_50" : 0,
    "X11" : 1,
    "whois" : 2,
    "vmnet" : 3,
    "uucp_path" : 4,
    "uucp" : 5,
    "urp_i" : 6,
    "urh_i" : 7,
    "time" : 8,
    "tim_i" : 9,
    "tftp_u" : 10,
    "telnet" : 11,
    "systat" : 12,
    "supdup" : 13,
    "sunrpc" : 14,
    "ssh" : 15,
    "sql_net" : 16,
    "smtp" : 17,
    "shell" : 18,
    "rje" : 19,
    "remote_job" : 20,
    "red_i" : 21,
    "private" : 22,
    "printer" : 23,
    "pop_3" : 24,
    "pop_2" : 25,
    "pm_dump" : 26,
    "other" : 27,
    "ntp_u" : 28,
    "nntp" : 29,
    "nnsp" : 30,
    "netstat" : 31,
    "netbios_ssn" : 32,
    "netbios_ns" : 33,
    "netbios_dgm" : 34,
    "name" : 35,
    "mtp" : 36,
    "login" : 37,
    "link" : 38,
    "ldap" : 39,
    "kshell" : 40,
    "klogin" : 41,
    "iso_tsap" : 42,
    "IRC" : 43,
    "imap4" : 44,
    "http_8001" : 45,
    "http_443" : 46,
    "http_2784" : 47,
    "http" : 48,
    "hostnames" : 49,
    "harvest" : 50,
    "gopher" : 51,
    "ftp_data" : 52,
    "ftp" : 53,
    "finger" :  54,
    "exec" : 55,
    "efs" : 56,
    "ecr_i" : 57,
    "eco_i" : 58,
    "echo" : 59,
    "domain_u" : 60,
    "domain" : 61,
    "discard" : 62,
    "daytime" : 63,
    "ctf" : 64,
    "csnet_ns" : 65,
    "courier" : 66,
    "bgp" : 67,
    "auth" : 68,
    "aol" : 69

}

flag_dict={
    "SH" : 0,
    "SF" : 1,
    "S3" : 2,
    "S2" : 3,
    "S1" : 4,
    "S0" : 5,
    "RSTR" : 6,
    "RSTOS0" : 7,
    "RSTO" : 8,
    "REJ" : 9,
    "OTH" : 10

}



df_test_datar.replace(flag_dict, inplace = True)
df_test_datar.replace(service_dict, inplace = True)
df_test_datar.replace(protocol_dict, inplace = True)
df_test_datar.replace(attack_dict, inplace = True)

df_test_datar.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,num_packet,type_attack
0,0,2,48,9,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,dos
1,0,1,48,9,64,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
2,0,2,48,9,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,normal
3,0,1,48,9,64,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1,1.0,1.00,0.00,0.0,3,250,1.00,0.00,0.33,0.12,0.0,0.0,1.0,1.0,21,normal
4,0,2,48,9,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,299,20,0.0,0.0,1,1.0,0.07,0.06,0.0,255,20,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0,20,normal


In [ ]:
X_test = df_test_datar.loc[:, :'dst_host_srv_rerror_rate']
y_test = df_test_datar.loc[:, 'type_attack']

y = y.astype(str)

temp = set(y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, train_size =0.5, random_state=5)
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state=5)


In [ ]:
clf = joblib.load('model_base.pkl')

In [ ]:
pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test,pred)

0.5676747645017353